In [ ]:
# installing pytorch lightning and transformer
! pip install --quiet lightning
! pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 24.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
# importing all necessary packages
from os import listdir
from os.path import join
from sklearn.model_selection import train_test_split
import string
from torch.utils.data import Dataset, DataLoader
import torch
import torchmetrics
import pytorch_lightning as pl
from transformers import AutoModelForSequenceClassification
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/BT5151/Group project'
    print(path_to_file)
    # move to Google Drive directory
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/BT5151/Group project
/content/gdrive/My Drive/BT5151/Group project


In [ ]:
%run distilbertroberta_tokenizer.py

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [ ]:
from distilbertroberta_tokenizer import tokenize_text_with_labels

In [ ]:
reviews_df = pd.read_csv('reviews_df.csv')

In [ ]:
reviews_df.head()

,ProductId,UserId,Score,Time_ymd,Summary_Text,Clean_Text,Original_Text_Length,Clean_Text_Length,Helpfulness_Ratio,Sentiment,NumUniqueProducts,Days_since_last_review,polarity,subjectivity
0,B001E4KFG0,A3SGXH7AUHU8GW,5,2011-04-27,Good Quality Dog Food I have bought several of...,good quality food bought several vitality cann...,285,162,1.0,1,1,248,0.480000,0.440000
1,B000UA0QIQ,A395BORC6FGVXV,2,2011-06-13,Cough Medicine If you are looking for the secr...,cough medicine looking secret ingredient robit...,234,139,1.0,-1,3,201,0.150000,0.650000
2,B000E7L2R4,A1MZYO9TZK0BBI,5,2011-11-23,Yay Barley Right now I'm mostly just sprouting...,barley right mostly sprouting cats grass love ...,142,70,1.0,1,2,37,0.428571,0.545238
3,B001GVISJM,A2A9X58G2GTBLP,5,2011-12-23,"GREAT SWEET CANDY! Twizzlers, Strawberry my ch...",great sweet candy twizzlers strawberry childho...,765,492,-1.0,1,44,8,0.271429,0.550000
4,B001GVISJM,A3IV7CL2C13K2U,5,2011-10-08,Home delivered twizlers Candy was delivered ve...,home delivered twizlers candy delivered fast p...,169,103,-1.0,1,1,84,0.225000,0.675000


In [ ]:
reviews_df.shape

(161599, 14)

# Distilroberta Transformer

In [ ]:
# Split the dataset into training, validation, and testing sets (80-10-10 ratio)
X_train, X_temp, y_train, y_temp = train_test_split(reviews_df['Summary_Text'], reviews_df['Sentiment'], test_size=0.2,shuffle=True,random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Create DataFrame for train data
train_df = pd.DataFrame({'Summary_Text': X_train, 'Score': y_train})

# Create DataFrame for test data
test_df = pd.DataFrame({'Summary_Text': X_test, 'Score': y_test})

# Assuming you have already split the test_df for validation and test set,
# you can proceed similarly to create DataFrame for validation data
val_df = pd.DataFrame({'Summary_Text': X_val, 'Score': y_val})

In [ ]:
train_df['Score'].unique()

array([ 1,  0, -1])

In [ ]:
# Check the sizes of the sets
print(f"Number of samples in the training set: {len(train_df)}")
print(f"Number of samples in the validation set: {len(val_df)}")
print(f"Number of samples in the testing set: {len(test_df)}")

Number of samples in the training set: 129279
Number of samples in the validation set: 16160
Number of samples in the testing set: 16160


In [ ]:
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

Tokenizer input max length: 512
Tokenizer vocabulary size: 50265


In [ ]:
# Instantiate the dataset objects

train_dataset = tokenize_text_with_labels(train_df, 512)
val_dataset = tokenize_text_with_labels(val_df, 512)
test_dataset = tokenize_text_with_labels(test_df, 512)

# Create DataLoader for each set
train_loader = DataLoader(
    dataset=train_dataset,
    persistent_workers = True,
    batch_size=32,
    shuffle=True,
    num_workers=10
)

val_loader = DataLoader(
    dataset=val_dataset,
    persistent_workers = True,
    batch_size=32,
    num_workers=10
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=32,
    num_workers=10
)


In [ ]:
train_dataset[0]

{'input_ids': tensor([    0,   565, 19626,    53,    24,   473,  5585, 39106,   152, 20437,
         18329,    16,   182, 22307,     8,  1153,    55,  2245,    87,   144,
            53,    24,   473,  5585, 39106,  7397,    11,     5,  1026,     9,
         31851, 14660,   479,    20,  2170,     9,     5,  1152,   855,   145,
           341,  2563,  4564,    22,  3084, 39106,   113,    16,    41,   793,
            65,   479,    20,  7508,    34,    55,   682, 21961,   196,    49,
          2026,     8,  2928,     5,    22,  3084, 39106,   113,    31,     5,
          9805,   479,     2,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [ ]:
# write your code here
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilroberta-base", num_labels=3)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [ ]:
# freeze all layer
for param in model.parameters():
    param.requires_grad = False
    # unfreeze last layer
for param in model.classifier.out_proj.parameters():
    param.requires_grad = True
for param in model.classifier.dense.parameters():
    param.requires_grad = True


In [ ]:
import numpy as np
np.random.seed(25)

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torchmetrics
from torchmetrics.classification import MulticlassPrecision, MulticlassRecall

class LightningModel(pl.LightningModule):
    def __init__(self, model, learning_rate=2e-3):
        super().__init__()

        self.learning_rate = learning_rate
        self.model = model ## use pretrained DistilBERT model

        self.val_f1 = torchmetrics.F1Score(task="multiclass", num_classes=3, average='weighted')
        self.val_precision = MulticlassPrecision(num_classes=3, average='weighted')
        self.val_recall = MulticlassRecall(num_classes=3, average='weighted')
        self.test_f1 = torchmetrics.F1Score(task="multiclass", num_classes=3, average='weighted')
        self.test_precision = MulticlassRecall(num_classes=3, average='weighted')
        self.test_recall = MulticlassRecall(num_classes=3, average='weighted')
        self.loss = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels):
        return self.model(input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["label"])
        self.log("train_loss", outputs["loss"])
        return outputs["loss"]  # this is passed to the optimizer for training

    def validation_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["label"])
        self.log("val_loss", outputs["loss"], prog_bar=True)
        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self.val_f1(predicted_labels, batch["label"])
        self.val_precision(predicted_labels, batch["label"])
        self.val_recall(predicted_labels, batch["label"])
        self.log_dict({"val_f1": self.val_f1, "val_precision": self.val_precision, "val_recall": self.val_recall}, on_step=False, on_epoch=True, prog_bar =True)


    def test_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["label"])
        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self.test_f1(predicted_labels, batch["label"])
        self.test_precision(predicted_labels, batch["label"])
        self.test_recall(predicted_labels, batch["label"])
        self.log_dict({"test_f1": self.test_f1, "test_precision": self.test_precision, "test_recall": self.test_recall}, on_step=False, on_epoch=True)


    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


lightning_model = LightningModel(model)


In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_f1",
        dirpath= '/content/gdrive/My Drive/BT5151/Group project',  # Directory to save the checkpoints
        filename='best_distilroberta_model_{val_f1:.2f}',
    )  # save top 1 model
]

In [ ]:
trainer = pl.Trainer(
    max_epochs=5,
    callbacks=callbacks,
    accelerator="gpu",
    devices=1,
    log_every_n_steps= 5000

)
trainer.fit(model=lightning_model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader)

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type                             | Params
--------------------------------------------------------------------
0 | model          | RobertaForSequenceClassification | 82.1 M
1 | val_f1         | MulticlassF1Score                | 0     
2 | val_precision  | MulticlassPrecision              | 0     
3 | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
trainer.test(lightning_model, dataloaders=test_loader, ckpt_path="best")

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/gdrive/My Drive/BT5151/Group project/best_distilroberta_model_val_f1=0.86.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/gdrive/My Drive/BT5151/Group project/best_distilroberta_model_val_f1=0.86.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          test_f1          │    0.8612757921218872     │
│      test_precision       │    0.8795791864395142     │
│        test_recall        │    0.8795791864395142     │
└───────────────────────────┴───────────────────────────┘

[{'test_f1': 0.8612757921218872,
  'test_precision': 0.8795791864395142,
  'test_recall': 0.8795791864395142}]

In [ ]:
trainer.validate(lightning_model, dataloaders=val_loader, ckpt_path="best")

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/gdrive/My Drive/BT5151/Group project/best_roberta_model_epoch=04-val_f1=0.88.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/gdrive/My Drive/BT5151/Group project/best_roberta_model_epoch=04-val_f1=0.88.ckpt


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_f1           │    0.8781498074531555     │
│         val_loss          │    0.28460028767585754    │
│       val_precision       │    0.8779380917549133     │
│        val_recall         │    0.8910890817642212     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.28460028767585754,
  'val_f1': 0.8781498074531555,
  'val_precision': 0.8779380917549133,
  'val_recall': 0.8910890817642212}]